<a href="https://www.kaggle.com/code/skshmjn/bert-ner?scriptVersionId=213846601" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [3]:
!pip install transformers datasets tokenizers seqeval evaluate -q 

In [20]:
import datasets 
import numpy as np 
import torch 
import json
from transformers import pipeline
from evaluate import load
from transformers import BertTokenizerFast
from transformers import DataCollatorForTokenClassification , DataCollatorWithPadding
from transformers import AutoModelForTokenClassification , AutoTokenizer
from transformers import TrainingArguments, Trainer 

In [5]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [7]:
conll2003 = datasets.load_dataset("conll2003", trust_remote_code=True)
conll2003

README.md:   0%|          | 0.00/12.3k [00:00<?, ?B/s]

conll2003.py:   0%|          | 0.00/9.57k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [8]:
conll2003["train"][0]

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

In [9]:
ner_classes = conll2003["train"].features["ner_tags"].feature.names
ner_classes

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [10]:
example = conll2003["train"][345]
example['tokens'] ,[ner_classes[i] for i in example['ner_tags']]

(['SOCCER', '-', 'GLORIA', 'BISTRITA', 'BEAT', '2-1', 'F.C.', 'VALLETTA', '.'],
 ['O', 'O', 'B-ORG', 'I-ORG', 'O', 'O', 'B-ORG', 'I-ORG', 'O'])

In [11]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [12]:
def tokenize_and_align_labels(examples, label_all_tokens=True): 

    # Tokenisation
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True) 
    labels = [] 

    # checking all the labels and adding label for word piece 
    for i, label in enumerate(examples["ner_tags"]):
        
        word_ids = tokenized_inputs.word_ids(batch_index=i) 
        previous_word_idx = None 
        label_ids = []
        
        for word_idx in word_ids: 
            if word_idx is None: 
               
                label_ids.append(-100)
           
            elif word_idx != previous_word_idx:
               
                label_ids.append(label[word_idx]) 
            else: 
               
                label_ids.append(label[word_idx] if label_all_tokens else -100) 
                 
            previous_word_idx = word_idx 
        labels.append(label_ids) 
    tokenized_inputs["labels"] = labels 
    return tokenized_inputs 

In [ ]:
result = tokenize_and_align_labels(conll2003['train'][345:346])
print(result)

In [ ]:
for token, label in zip(tokenizer.convert_ids_to_tokens(result["input_ids"][0]),result["labels"][0]): 
    print(f"{token:_<40} {label}") 

In [13]:
tokenized_datasets = conll2003.map(tokenize_and_align_labels, batched=True)


Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(["tokens","pos_tags", "chunk_tags", "ner_tags"])

In [ ]:
tokenized_datasets['train'][0:3]

In [ ]:
model = AutoModelForTokenClassification.from_pretrained("bert-base-uncased", num_labels=9).to(device)

In [27]:
data_collator = DataCollatorForTokenClassification(tokenizer) 
metric = load("seqeval") 

In [22]:
def compute_metrics(eval_preds): 
 
    pred_logits, labels = eval_preds 
    
    pred_logits = np.argmax(pred_logits, axis=2) 
    
    predictions = [ 
        [ner_classes[eval_preds] for (eval_preds, l) in zip(prediction, label) if l != -100] 
        for prediction, label in zip(pred_logits, labels) 
    ] 
    
    true_labels = [ 
      [ner_classes[l] for (eval_preds, l) in zip(prediction, label) if l != -100] 
       for prediction, label in zip(pred_logits, labels) 
   ] 
    results = metric.compute(predictions=predictions, references=true_labels) 
    return { 
   "precision": results["overall_precision"], 
   "recall": results["overall_recall"], 
   "f1": results["overall_f1"], 
  "accuracy": results["overall_accuracy"], 
  } 

In [25]:
args = TrainingArguments( 
"test-ner",
evaluation_strategy = "epoch", 
learning_rate=2e-5, 
per_device_train_batch_size=64, 
per_device_eval_batch_size=64, 
num_train_epochs=5, 
weight_decay=0.01, 
) 

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [23]:
trainer = Trainer( 
    model, 
    args, 
   train_dataset=tokenized_datasets["train"], 
   eval_dataset=tokenized_datasets["validation"], 
   data_collator=data_collator, 
   tokenizer=tokenizer, 
   compute_metrics=compute_metrics 
) 

NameError: name 'model' is not defined

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate(tokenized_datasets["test"])

In [ ]:
model.save_pretrained("ner_model")
tokenizer.save_pretrained("tokenizer")

In [ ]:
id2label = {
    str(i): label for i,label in enumerate(ner_classes)
}
label2id = {
    label: str(i) for i,label in enumerate(ner_classes)
}

config = json.load(open("ner_model/config.json"))
config["id2label"] = id2label
config["label2id"] = label2id
json.dump(config, open("ner_model/config.json","w"))

In [ ]:
model_fine_tuned = AutoModelForTokenClassification.from_pretrained("ner_model").to(device)
nlp = pipeline("ner", model=model_fine_tuned, tokenizer=tokenizer)

example = "Narendra Damodardas Modi[a] (born 17 September 1950)[b] is an Indian politician who has served as Prime Minister of India since 2014. Modi was the chief minister of Gujarat from 2001 to 2014 and is the member of parliament (MP) for Varanasi. He is a member of the Bharatiya Janata Party (BJP) and of the Rashtriya Swayamsevak Sangh (RSS), a right-wing Hindu nationalist paramilitary volunteer organisation. He is the longest-serving prime minister outside the Indian National Congress."

ner_results = nlp(example)

print(ner_results)

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!sudo apt-get install git-lfs

In [ ]:
trainer.push_to_hub()

## Qunatisation

In [21]:
model_fine_tuned = AutoModelForTokenClassification.from_pretrained("skshmjn/test-ner").to(device)
tokenizer = AutoTokenizer.from_pretrained('skshmjn/test-ner')

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [15]:
quantized_model = torch.quantization.quantize_dynamic(
    model_fine_tuned, {torch.nn.Linear}, dtype=torch.qint8
)
print(quantized_model)

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): DynamicQuantizedLinear(in_features=768, out_features=768, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
              (key): DynamicQuantizedLinear(in_features=768, out_features=768, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
              (value): DynamicQuantizedLinear(in_features=768, out_features=768, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
              (dropout): Dropout(p=0.1, inplace=False)
           

In [16]:
def print_size_of_model(model):
    torch.save(model.state_dict(), "temp_delme.p")
    print('Size (KB):', os.path.getsize("temp_delme.p")/1e3)
    os.remove('temp_delme.p')


In [17]:
print_size_of_model(model_fine_tuned)
print_size_of_model(quantized_model)

Size (KB): 435672.965
Size (KB): 180905.751


In [28]:
fine_tune_trainer = Trainer( 
    model_fine_tuned, 
    args, 
   train_dataset=tokenized_datasets["train"], 
   eval_dataset=tokenized_datasets["validation"], 
   data_collator=data_collator, 
   tokenizer=tokenizer, 
   compute_metrics=compute_metrics 
) 

/tmp/ipykernel_24/1340368719.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  fine_tune_trainer = Trainer(


In [30]:
quntised_trainer = Trainer( 
    quantized_model, 
    args, 
   train_dataset=tokenized_datasets["train"], 
   eval_dataset=tokenized_datasets["validation"], 
   data_collator=data_collator, 
   tokenizer=tokenizer, 
   compute_metrics=compute_metrics 
) 

/tmp/ipykernel_24/2746637661.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  quntised_trainer = Trainer(


In [29]:
fine_tune_trainer.evaluate(tokenized_datasets["test"])

{'eval_loss': 0.10942327231168747,
 'eval_model_preparation_time': 0.005,
 'eval_precision': 0.8816678313533659,
 'eval_recall': 0.8984096843104676,
 'eval_f1': 0.8899600282153775,
 'eval_accuracy': 0.9746219183758028,
 'eval_runtime': 306.8166,
 'eval_samples_per_second': 11.254,
 'eval_steps_per_second': 0.176}

In [31]:
quntised_trainer.evaluate(tokenized_datasets["test"])

{'eval_loss': 0.18233591318130493,
 'eval_model_preparation_time': 0.0047,
 'eval_precision': 0.8597059556441565,
 'eval_recall': 0.8188938998338476,
 'eval_f1': 0.838803792851933,
 'eval_accuracy': 0.9648504937504316,
 'eval_runtime': 225.5239,
 'eval_samples_per_second': 15.311,
 'eval_steps_per_second': 0.239}